In [18]:
# nbi:hide_in
import ipywidgets as widgets
from ipywidgets import interactive
import datetime as dt
import matplotlib.pyplot as plt
import yfinance as yf
import matplotlib.dates as md
import matplotlib as mpl
import math
import numpy as np
import pandas as pd
import warnings
import scipy
from scipy.optimize import minimize
from datetime import datetime
warnings.filterwarnings("ignore")


RSI_PERIOD=14


SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


def get_data(stock,start,end,interval):
    alldata = yf.download(stock,start=start,end=end, interval=interval)

    return alldata

def analyze(stock,money,data,split=0.5,alldata=[],nlevels=4):
    
    alldata=compute_HODL(data,money)
    df = alldata.iloc[:int(len(alldata)*split)] 
    df2 = alldata.iloc[int(len(alldata)*split):] 

    supp1,resist1,df1,summary1=calculate(df,nlevels)
    print(summary1)
    supp2,resist2,df2,summary2=calculate(df2,nlevels,calc_level=0,levels=list(summary1.index))
    
    adata,support, resist=combine(supp1,supp2,resist1,resist2,df1,df2)
    plot(stock,adata,support, resist,list(summary1.index),int(len(alldata)*split))
    
    earns_display(stock,df1,adata,money,summary1,split)
    
    return adata,summary1


def compute_HODL(data,money):
    data['reg_earn']=data['Close'].apply(lambda x: money*(1+(x-money)/money))
    data['HODL']=data['reg_earn'].apply(lambda x: 100*(x-data['reg_earn'].iloc[0])/data['reg_earn'].iloc[0])
    return data

def calculate(data,nlevels=4,money=1,calc_level=1,levels=[]):
    
    #Levels and categorize
    if calc_level:
        levels=get_levels(data['High'].max(),data['Low'].min(),nlevels)
    
    data['Levels'] = data['Close'].apply(lambda x: classify(x,levels))   
    if data['Levels'].max()== len(levels):
        pmax=data['Close'].max()
        levels.append(pmax)
    
    #We correct the index and compute Change in Levels
    data['Levels']=(data['Levels'].fillna(1)-1).astype(int) 
    
    #Compute Gradient and 2nd Derivative
    data['gradient']=data['Close'].diff()
    data['acc']=  data['gradient'].diff()
    data['Force'] = (data.acc)

    data['kdivs'] = abs(data.Force.div((data['Levels'].apply(lambda x: levels[x+1] if x<len(levels)-1 else levels[x])-(data['Levels'].apply(lambda x: levels[x])))*0.1 +data['Levels'].apply(lambda x: levels[x]) -data['Close']))
    data['kdivr'] = abs(data.Force.div(data['Close']-((data['Levels'].apply(lambda x: levels[x+1] if x<len(levels)-1 else levels[x])-(data['Levels'].apply(lambda x: levels[x])))*0.9 +data['Levels'].apply(lambda x: levels[x]))))
    
    support=data[(data.Close>=data['Levels'].apply(lambda x: levels[x]))  & (data.acc>0)]  #
    resist=data[(data.Close<=data['Levels'].apply(lambda x: levels[x+1] if x<len(levels)-1 else levels[x]))  &(data.acc<0)]  #
    resist.loc[:,'Force']=resist['Force'].apply(lambda x: -x)
    
    krest=resist.groupby('Levels')['kdivr'].mean().to_frame('Value')
    krest=krest['Value'].reindex(range(len(levels)), fill_value= 0)
    krest=np.round(krest.shift(1).replace(np.nan,0),3)
    
    krestdev=resist.groupby('Levels')['kdivr'].std().to_frame('Value')
    krestdev=krestdev['Value'].reindex(range(len(levels)), fill_value= 0)
    krestdev=np.round(krestdev.shift(1).replace(np.nan,0),3)
 
    restmax=resist.groupby('Levels')['Force'].max().to_frame('Value')
    restmax=restmax['Value'].reindex(range(len(levels)), fill_value= 0)
    restmax=np.round(restmax.shift(1).replace(np.nan,0),3)
    
    rest=resist.groupby('Levels')['Force'].mean().to_frame('Value')
    rest=rest['Value'].reindex(range(len(levels)), fill_value= 0)
    rest=np.round(rest.shift(1).replace(np.nan,0),3)
  
    ksupp=support.groupby('Levels')['kdivs'].mean().to_frame('Value')
    ksupp=np.round(ksupp['Value'].reindex(range(len(levels)), fill_value= 0),3)
    ksuppdev=support.groupby('Levels')['kdivs'].std().to_frame('Value')
    ksuppdev=np.round(ksuppdev['Value'].reindex(range(len(levels)), fill_value= 0),3)
    suppmax=support.groupby('Levels')['Force'].max().to_frame('Value')
    suppmax=np.round(suppmax['Value'].reindex(range(len(levels)), fill_value= 0),3)
    supp=support.groupby('Levels')['Force'].mean().to_frame('Value')
    supp=np.round(supp['Value'].reindex(range(len(levels)), fill_value= 0),3)
    
    
    F=pd.DataFrame(columns=['Price','Support Mean','Support Max','Resistance Mean','Resistance Max','SConst','RConst','Sdev','Rdev'])
    F['Price']=levels

    F['Support Mean']= supp
    F['SConst']=ksupp
    F['Resistance Mean']=rest
    F['Support Max'] =suppmax
    F['Resistance Max'] = restmax
    F['RConst']=krest
    F['Sdev']=ksuppdev
    F['Rdev']=krestdev
    F=F.set_index('Price')
    
    resist.loc[:,'Force']=resist['Force'].apply(lambda x: -x)
    
    out=support,resist,data,F
    
    return out


def get_levels(price_max,price_min,nlevels):
    diff = price_max - price_min
    sequences=[0,1]
    nterms=7+nlevels+1
    n1, n2 = 0, 1
    count = 2

    while count < nterms:
        sequences.append(sequences[count-2]+sequences[count-1])
        count=count+1
    del sequences[0:7]


    count=0
    levels=[price_min]
    while count < nlevels:
            levels.append(price_max - round(sequences[0]/sequences[count+1],3)*diff)
            count=count+1
    levels.append(price_max)
    
    return levels


def classify(value,levels):
    level=-1
    for i in range(0,len(levels)):
        if value <= levels[i]:
            level = i
            break
    if level==-1:
        level = len(levels)
    return level

def combine(supp1,supp2,resist1,resist2,df1,df2):
    support = pd.concat([supp1,supp2]).sort_index()  
    resist = pd.concat([resist1,resist2]).sort_index()  
    adata=pd.concat([df1,df2])
    return adata,support, resist


def orderofmagnitude(number):
    try:
        return math.floor(math.log(number, 10))
    except:
        return 0

def plot(stock,data,s,r,levels,n):

    fig, ax = plt.subplots()
    end1=data.index[n]

    fig.subplots_adjust(hspace=0.6)
    plt.gcf().set_size_inches(15,8)
    top=plt.subplot2grid((7,5),(0,0),rowspan=4,colspan=5)
    top.plot(data.index,data['Close'],label=stock + ' Close Price',color='black')
    top.axvline(x=end1,ls='--',color='blue')
    
    top.scatter(data.index,data['Close'],label=stock + ' Close Price',color='black',marker='o')
    for i in range(0,len(levels)):
        top.axhline(levels[i],ls='--',color='black',alpha=0.4)
    
    par1 = top.twinx() #Support
    par2 = top.twinx() #Resistance
    
    par1.bar(s.index,s.Force,label='Support',color='green',alpha=0.6)
    par2.bar(r.index,r.Force,label='Resistance',color='red',alpha=0.6)
    par1.spines['right'].set_position(('outward', 80))
    
    par1.set_ylabel("Support")
    par2.set_ylabel("Resistance")

    par1.yaxis.label.set_color('green')
    par2.yaxis.label.set_color('red')
 
    ymin,ymax=par1.get_ylim()   #Support    
    inc=orderofmagnitude(ymax)
    par1.set_ylim([ymin,ymax+(10**inc)/2])  
    
    
    ymin,ymax=par2.get_ylim()
    inc=orderofmagnitude(abs(ymin))
    par2.set_ylim([ymin-(10**inc)/2,ymax])  #Resistance
    
    ymin,ymax=top.get_ylim()
    inc=orderofmagnitude(ymax)    

    top.set_ylabel(stock + " Close Price ($)")
    top.set_xlabel("Date")
    
    top.set_xlim(data.index[0],data.index[-1])
    
    return



def earns_display(stock,df1,adata,money,summary1,split):

    earn1=get_earning(df1,money,summary1)
    earnf=get_earning(adata,money,summary1)
    plot_earn(stock,earn1,1)
    plot_earn(stock,earnf,split)
    
    return


def get_earning(data,money,thresholds):
        levels=list(thresholds.index)
        
        base=data['Close'].iloc[0]
    
        data['dyn_earn']=money
        data['action'] = data.apply(lambda x: 1 if x.Force>0 else -1 ,axis=1)
        data['daction'] = data['action'].diff().replace(np.nan,-10)

        for i in np.arange(1,len(data)):
            data['dyn_earn'].iloc[i]=money*(1+data['action'].iloc[i-1]*(data['Close'].iloc[i]-base)/base)              
            if data['daction'].iloc[i]!=0: #close and changing position
                money = data['dyn_earn'].iloc[i]
                base = data['Close'].iloc[i]

        data['Villagracia']=data['dyn_earn'].apply(lambda x: 100*(x-data['dyn_earn'].iloc[0])/data['dyn_earn'].iloc[0])

        return data
    

def plot_earn(stock,data,split):
    fig,ax = plt.subplots(figsize=(15,6))
    data[['HODL','Villagracia']].plot(ax=ax,rot=0)
    ymin,ymax=ax.get_ylim()
    ax.set_ylabel('returns(%)')
    ax.text(data.index[int(len(data)*0.8/2)],ymin+(ymax-ymin)*0.90,'Villagracia: ' + str(np.round(data['Villagracia'].iloc[len(data)-1],2))+'%')
    ax.text(data.index[int(len(data)*0.8/2)],ymin+(ymax-ymin)*0.85,'HODL: ' + str(np.round(data['HODL'].iloc[len(data)-1],2))+'%')
    ax.axvline(data.index[int(len(data)*split)-1],ls='--',color='blue',alpha=0.4)
    ax.set_title(str(stock) + ' Returns')
    ax.set_xlim([data.index[0],data.index[-1]])
    
    return


def execute(stock='AAPL', start='03-01-2021', end='06-09-2021',interval='1h', split=0.8, nlevels=4,money=100):

    data_aapl=get_data(stock,start,end,interval)
    adata,summary=analyze(stock,money,data_aapl,split=split,nlevels=nlevels)

def run():

    out=    interactive(execute,stock=widgets.Dropdown(options=['AAPL', 'BTC-USD']),
                     start=widgets.DatePicker(value = dt.date(2021,1,1),description='Start Date'),
                     end=widgets.DatePicker(value = dt.date.today(),description='End Date'),
                     interval=widgets.Dropdown(options=['1m', '1h', '1d'],value='1h',description='Data interval:'),
                     split=widgets.BoundedFloatText(value=0.8,min=0.5,max=0.9,step=0.05,description='Split Ratio:'),
                     nlevels=widgets.BoundedIntText(value=4,min=4,max=10,step=1,description='Levels:'),
                     money=widgets.BoundedIntText(value=100,min=100,max=1000,step=1,description='Investment:')
                    )
    return out

run()

interactive(children=(Dropdown(description='stock', options=('AAPL', 'BTC-USD'), value='AAPL'), DatePicker(val…